# Интерфейсы

Интерфейс - контракт, по которому класс, его реализующий, предоставляет какие-то методы.

Написание кода с опорой на интерфейсы, а не на конкретные типы позволяет:
- **Переиспользовать код, абстрагируясь от реализации.** Один раз написанный алгоритм сортировки элементов, опирающийся только на интерфейс IComparable, одинаково работает как со встроенными типами, так и с вашими.
- **Подменять реализацию, в том числе во время исполнения.**
- **Сделать код более безопасным.** Объект, передаваемый по интерфейсной ссылке предоставляет только ограниченную информацию о своих возможностях.
- **Не опасаться за последствия (по сравнению с наследованием).** Так как мы не тянем за собой реализацию, не возникает проблем, как с множественным наследованием.

## 1. Правила определения интерфейсов

В интерфейсе определяются сигнатуры *экземплярных функциональных* членов класса, кроме конструкторов. 

Т.е. недопустимы
- Поля
- Конструкторы

Всё остальное - можно:
- Методы
- Свойства
- События
- Индексаторы

Начиная с C# 8.0 (кажется) можно определять в интерфейса *статические и экземплярные методы с реализацией*. 

Модификатор доступа не указывается - он априори public.

In [ ]:
public interface ISomethingMessy
{
    // Метод
    void Execute();
    
    // Свойство
    string Message { get; }
    
    // Индексатор
    object this[int index] { get; set; }
    
    // Событие
    event Action MyEvent;
    
    // Лучше не переходить эту черту...
    // --------------------------------
    
    // Статический метод - обязательна реализация
    static void StaticMethod() 
    { 
        Console.WriteLine("interface static method");
    }
    
    // Дефолтная реализация интерфейса: ДОСТУПНА ТОЛЬКО ПО ИНТЕРФЕЙСНОЙ ССЫЛКЕ 
    void SecretMethod()
    {
        Console.WriteLine("Your password is 123456");
    }
}

Пример из стандартной библиотеки - System.IDisposable
```csharp
public interface IDisposable
{
    void Dispose();
}
```

## 2. Реализация интерфейсов. Наследование

In [ ]:
using System.IO;

class Base : IDisposable
{
    private FileStream fileStream;
    
    // ...
    
    // public void Dispose() { fileStream.Dispose(); }
}

In [ ]:
using System.IO;

class Base : IDisposable
{
    private FileStream fileStream;
    
    // ...
    
    public void Dispose() { fileStream.Dispose(); }
}

class Derived : Base
{
    // ...
}

// Все наследники класса автоматически реализуют интерфейсы родителя.
Derived derived = new Derived();
derived is IDisposable

## 3. Также доступны методы класса object

In [ ]:
IComparable<int> val = 3;
val.ToString()

In [ ]:
val.GetType()

## 4. ~~Реализация~~ Наследование интерфейсов интерфейсами

Можно расширить интерфейс, отнаследовав от него другой интерфейс. Типы, реализующие интерфейс-ребёнок будут обязаны реализовать функционал обоих интерфейсов.

**Однако это оправдано тогда и только тогда, когда жёсткая связь допустима.**

Иначе лучше использовать несколько маленьких интерфейсов согласно **Interface Segregation Principle**.

In [ ]:
public interface IVehicle
{
    void MoveTo(float x, float y, float z);
}

public interface IWheeledVehicle : IVehicle
{
    int NumOfWheels { get; }
}

public class Car : IWheeledVehicle { }


Пример наследования интерфейсов из стандартной библиотеки - IEnumerable

```csharp
public interface IEnumerable<out T> : IEnumerable
{
    IEnumerator<T> GetEnumerator();
}

public interface IEnumerable
{
    IEnumerator GetEnumerator();
}
```

## 5. Явная (explicit) и неявная (implicit) реализации интерфейса

Однако можно реализовать интерфейс, не предоставив публичную реализацию методов.

Этого можно добиться, реализовав интерфейс **явно** (explicit). Такая реализация будет доступна **только по соответствующей интерфейсной ссылке**.

In [ ]:
public class MyClass : IDisposable
{
    // Неявная реализация интерфейса
    // public void Dispose() { Console.WriteLine("Implicit"); }

    // Явная реализация интерфейса
    void IDisposable.Dispose() { Console.WriteLine("Explicit"); }
}

In [ ]:
MyClass myClass = new MyClass();
myClass.Dispose();

In [ ]:
IDisposable disposable = new MyClass();
disposable.Dispose();

**В чём смысл?**

Можно реализовать несколько интерфейсов, содержащих несколько одинаковых по сигнатуре методов. Если они представляют одинаковый смысл то проблем не возникает - а если они в сущности разные?

С помощью явных реализаций интерфейса можно определить **разное поведение** экземпляра в зависимости от того, по какой ссылке мы вызываем интерфейсный метод.

P.S. Пример супер надуманный, как обычно

In [ ]:
// "Исполнитель"
public interface IExecutor
{
    void Execute();
}

In [ ]:
// "Палач"
public interface IExecutioner
{
    void Execute();
}

In [ ]:
public class Officer : IExecutor, IExecutioner
{
    public void Execute() { /* some boring actions */ Console.WriteLine("Job executed."); }
    
    void IExecutioner.Execute() { /* some murderous actions */ Console.WriteLine("Intruder executed."); }
}

In [ ]:
Officer officer = new Officer();
officer.Execute();

In [ ]:
IExecutor executor = officer;
executor.Execute();

In [ ]:
IExecutioner executioner = officer;
executioner.Execute();

## 6. Обобщённые интерфейсы

Интерфейсы могут быть обобщёнными, таким образом получив все преимущества обобщений.

Из приятного: можно реализовать один и тот же интерфейс с различными параметрами типа, т.к. *как вы знаете*, обобщённые типы с разными параметрами конструируются в разные типы.

In [ ]:
public class Number : IComparable<int>, IComparable<double>, IComparable<string>
{
    private int Value { get; }
    
    public Number(int number)
    {
        Value = number;
    }

    public int CompareTo(int other) 
    {
        Console.WriteLine("Hello from int");
        return Value.CompareTo(other);
    }
    
    
    public int CompareTo(double other)
    {
        Console.WriteLine("Hello from double");
        return ((double)Value).CompareTo(other);
    }
    
    public int CompareTo(string other)
    {
        Console.WriteLine("Hello from string");
        return ((double)Value).CompareTo(double.Parse(other));
    }
}

In [ ]:
Number number = new Number(42);

In [ ]:
number.CompareTo(13)

In [ ]:
number.CompareTo(42.5)

In [ ]:
number.CompareTo("42")

Можно использовать интерфейсы в ограничениях на аргумент-тип. Если использовать несколько, то аргумент-тип должен реализовать все.

In [ ]:
public void SayHello<T>(T value) where T : IComparable<int>, IDisposable
{
    Console.WriteLine("Hello!");
}

In [ ]:
public class MyClass : IComparable<int> //, IDisposable
{
    public int CompareTo(int other) => throw new NotImplementedException();
    
    public void Dispose() => throw new NotImplementedException();
}

In [ ]:
MyClass obj = new MyClass();
SayHello(obj)

## 7. Реализация метода интерфейса по умолчанию

Начиная с C# 8.0 можно определять реализацию методов интерфейса по умолчанию.

Такая реализация доступна только по интерфейсной ссылке

In [ ]:
public interface ISummator
{
    int Sum(IEnumerable<int> values) 
    {
        int result = 0;
        foreach(var value in values)
        {
            result += value;
        }
        return result;
    }
}

In [ ]:
public class MySummator : ISummator
{
    // Можно переопределить, тогда конкретная реализация полностью перекроет дефолтную
    //public int Sum(IEnumerable<int> values) => values.Count();
}

In [ ]:
MySummator mySummator = new MySummator();

mySummator.Sum(new int[]{1, 2, 3, 4, 5})

In [ ]:
ISummator summator = new MySummator();

summator.Sum(new int[] { 1, 2, 3, 4, 5 })

## 8. Абстрактный класс или интерфейс?

**Абстрактный класс:**
- Является классом, а значит наследуясь от него нельзя наследоваться от других классов;
- Может определять часть состояния и поведения;
- Наследование - очень сильная связь;

Абстрактный определяет каркас для нескольких различных реализаций сущности.

**Интерфейс:**
- Класс может реализовывать сколько угодно интерфейсов;
- Определяет (в общем случае) только *что* должен делать класс, но не *как* (в общем случае);
- Реализация интерфейс - слабая связь;

Интерфейс определяет набор свойств, которыми должна обладать сущность, её некоторый обособленный функционал.